In [ ]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import math
# plt.style.use('dark_background')
from mog_fit import data
from collections import OrderedDict
from sklearn import preprocessing

import torch
from torch.autograd import Variable
from mog_analyze import viz
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

from scipy.stats import entropy
from scipy.stats import ks_2samp

%load_ext autoreload
%autoreload 2

# load in imagenet dset

In [ ]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

# Data loading code
data_dir = '/scratch/users/vision/data/cv/imagenet_full'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

batch_size = 10
num_workers = 1
'''
train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)
'''
val_loader = torch.utils.data.DataLoader(
    dset.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize])),
    batch_size=batch_size, shuffle=False, 
    num_workers=num_workers, pin_memory=True)

# load in some models

In [ ]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)

In [ ]:
alexnet.state_dict().keys()

In [ ]:
for x in val_loader:
    print(x.shape)
    break

In [ ]:
im = x[0][1].numpy().transpose(1, 2, 0)

In [ ]:
plt.imshow(im)
plt.show()

In [ ]:
alexnet(Variable(x[0])).shape

# registering forward hooks

In [ ]:
outputs= []
def hook(module, input, output):
    outputs.append(output)

res50_model = models.resnet50(pretrained=True)
res50_model.layer4[0].conv2.register_forward_hook(hook)
out = res50_model(Variable(x[0]))
# out = res50_model(res1)
print(len(outputs))

In [ ]:
o = outputs[0].shape

In [ ]:
o